# Load and initialize

settings

In [ ]:
# sample number
j=str(12456789)
singleSample = False
# n_samples = 10000
    # create visualization
visualize = True
    # show figures
show = True

# load_from_file = False
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
dir_data = parent_dir+'/Data/'
# dir_data = parent_dir+'/Data_sampNormScale/'
# dir_data = parent_dir+'/Data_sampScale/'

envirement

In [ ]:
import os

import time 
start = time.process_time()
import pathlib
import sys
from IPython.display import Image, display
from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context

plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"
%matplotlib inline

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

# from scipy import signal, stats
# import seaborn as sns

# from sklearn import metrics

# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

# from lmfit import minimize, Parameters


sys.path.append(parent_dir+'/functions/')


from usefull_functions import *
from functions import *
from plot_functions import *
from clustering_functions import *
# from epinuc import *



%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%autoreload 2


dir_plots = parent_dir+'/plots_sample'+j+'/'
saveSVG  =False
settings =        (dir_plots,show,saveSVG)
folderExists(dir_plots)

TF_ENABLE_ONEDNN_OPTS=0

load data; normalized (CD45-)  

In [ ]:
k_orig = pickle_load('k'+j+'_dict',dir_data )['k']
print(len(k_orig))

dict = pickle_load('k'+j+'_names_dict',dir_data )
NamesAll = dict['NamesAll']
CellIden = dict['CellIden']
EpiCols = dict['EpiCols']
Core = dict['Core']
CellIden_EpiCols = CellIden + EpiCols


groups = [['CellIden','EpiCols','CellIden+EpiCols'],[CellIden,EpiCols,CellIden_EpiCols]]
del dict


Visualize data

no point to print data.describe() as it is whitened (std~1, mean~0) 

In [ ]:
if show:

    for name in [NamesAll,Core,CellIden,EpiCols,CellIden_EpiCols]:
        print(len(name))
        print(name)


# Cell Identity

In [ ]:
k = k_orig.copy()
CAll=pd.concat([k]).copy()
s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}


name = 'Tumor'+j+'_CellIden_'


### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['1'],s['n_neighbors']['1'] = 0.001,10
s['min_dist']['6'],s['n_neighbors']['6'] = 0.001,20
s['min_dist']['7'],s['n_neighbors']['7'] = 0.001,10
s['min_dist']['8'],s['n_neighbors']['8'] = 0.001,10
s['min_dist']['9'],s['n_neighbors']['9'] = 0.001,10
s['min_dist']['12456789'],s['n_neighbors']['12456789'] = 0.001,10
s['min_dist']['456789'],s['n_neighbors']['456789'] = 0.001,10



s['min_dist']['245'],s['n_neighbors']['245'] = 0.001,50
s['min_dist']['1245'],s['n_neighbors']['1245'] = 0.01,50


min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]



umapData = calculate_umap(CAll[CellIden],n_neighbors, min_dist)
if visualize:
  

  drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')

# umap_params(k,CellIden,dir_plots,
#             valA = min_dist, valB = n_neighbors
#             )

# if j=='4':
#     ind  = np.where(umapData[:,0]<5)
#     k_orig.drop(k_orig.index[ind],inplace=True)
#     k = k_orig.copy();CAll=pd.concat([k]).copy()
    
    # umapData = calculate_umap(CAll[CellIden],n_neighbors, min_dist)
    # drawUMAP(umapData, ['H4'],CAll,settings,
    #             title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
    #             Figname = name+'1_UMAP_(core)')
    


### Dbscan and clusters

In [ ]:
s['min_samples']['1'],s['eps']['1']=50,0.19
s['min_samples']['6'],s['eps']['6']=50,0.19
s['min_samples']['7'],s['eps']['7']=50,0.19
s['min_samples']['8'],s['eps']['8']=50,0.13
s['min_samples']['9'],s['eps']['9']=50,0.19
s['min_samples']['12456789'],s['eps']['12456789']=70,0.1
s['min_samples']['456789'],s['eps']['456789']=70,0.1


s['min_samples']['245'],s['eps']['245']=50,0.19
s['min_samples']['1245'],s['eps']['1245']=140,0.18


eps=s['eps'][j];min_samples=s['min_samples'][j]
dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps,min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
              
  


# dbscan_params(X_2d,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


In [ ]:

plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Cell Iden Based',
             figname = name+'1_Clusters')
             



### HeatMap by clusters

In [ ]:
### HeatMap by clusters
            
if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        group_name,group = groups[0][-1],groups[1][-1]
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Cell Iden Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)


### HeatMap by sample

In [ ]:
if visualize and not singleSample :
    for group_name,group in zip(groups[0],groups[1]):
        group_name,group = groups[0][-1],groups[1][-1]
        HeatMap(k[k.Clust!=-1],group,settings,clustFeature='by_sample',
                title = 'T'+j+' Cell Iden Based: '+group_name+'by_sample',
                figname = name+'1_HeatMap_'+group_name+'by_sample')


### MeanDist

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist CellIdentity based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## sample percentage

In [ ]:
if visualize and not singleSample:
    arr =[]
    # full map
    ind = k.Clust != -1
    _ = drawUMAP_intVals(umapData,k,ind, 'by_sample',settings,
            title ='Tumor '+j+' Celliden UMAP', 
            Figname = name+'4_UMAP_(samples)')
    # map of each cluster
    clustNums = np.unique(k.Clust); clustNums = clustNums[clustNums!=-1]
    for clustNum in clustNums:
        ind = k.Clust == clustNum
        samplesPercentage = drawUMAP_intVals(umapData,k,ind ,'by_sample',settings,
            title =name+' UMAP clust' + str(clustNum), 
            Figname = name+'4_UMAP_clust'+str(clustNum)+'_(samples)')
        arr.append([clustNum,samplesPercentage])
    saveCsv(dir_plots,name,arr)

## features on UMAP

In [ ]:
if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor'+j+ ' Cell Iden UMAP', 
            Figname = name+'2_UMAP_')

## save data to file

In [ ]:
dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None

# Epigenetics Based

In [ ]:


k = k_orig.copy()
CAll=pd.concat([k]).copy()

s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}

name = 'Tumor'+j+'_EpiCols_'



### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['1'],s['n_neighbors']['1'] = 0.07,200
s['min_dist']['6'],s['n_neighbors']['6'] = 0.001,10
s['min_dist']['7'],s['n_neighbors']['7'] = 0.001,10
s['min_dist']['8'],s['n_neighbors']['8'] = 0.001,10
s['min_dist']['9'],s['n_neighbors']['9'] = 0.001,10
s['min_dist']['12456789'],s['n_neighbors']['12456789'] = 0.001,200
s['min_dist']['456789'],s['n_neighbors']['456789'] = 0.001,10

s['min_dist']['245']=0.001
s['n_neighbors']['245']=10
s['min_dist']['1245']=0.001
s['n_neighbors']['1245']=10

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]


umapData = calculate_umap(CAll[EpiCols],n_neighbors, min_dist)
if visualize:
    drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')
    


# umap_params(k,MRK,dir_plots,
#             valA = min_dist, valB = n_neighbors,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.2, valB = 300,
#             )
# umap_params(k,MRK,dir_plots,
#             valA = 0.1, valB = 300,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.001, valB = 300,
#             )

### Dbscan and clusters

In [ ]:
s['min_samples']['1'],s['eps']['1']=80,0.4
s['min_samples']['6'],s['eps']['6']=50,0.14
s['min_samples']['7'],s['eps']['7']=50,0.19
s['min_samples']['8'],s['eps']['8']=50,0.17
s['min_samples']['9'],s['eps']['9']=50,0.16
s['min_samples']['12456789'],s['eps']['12456789'] = 50,0.095
s['min_samples']['456789'],s['eps']['456789']=20,0.05

s['min_samples']['245']=50
s['eps']['245']=0.19
s['min_samples']['1245']=100
s['eps']['1245']=0.2
min_samples = s['min_samples'][j]
eps = s['eps'][j]


dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps=eps,min_samples=min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
            
# dbscan_params(X_2d,dir_plots,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


In [ ]:
plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Epigen Based',
             figname = name+'1_Clusters')


## HeatMap by cluster

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
    
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Epigen Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)


### HeatMap by sample

In [ ]:

if visualize and not singleSample:

    group_name,group = groups[0][-1],groups[1][-1]
    HeatMap(k[k.Clust!=-1],group,settings,clustFeature='by_sample',
            title = 'T'+j+' Cell Iden Based: '+group_name+'by_sample',
            figname = name+'1_HeatMap_'+group_name+'by_sample')


epinuc

In [ ]:
# Kepinuc = loadEpinuc("EPINUC_BCK.csv",sampleMax=5)

# # group_name,group = groups[0][-1],groups[1][-1]
# # HeatMap(k,group,settings,clustFeature='by_sample',
# #         title = 'T'+j+' Cell Iden Based: '+group_name+'by_sample',
# #         figname = name+'1_HeatMap_'+group_name+'by_sample')
# Mat=   k.groupby(by='by_sample').mean()[Kepinuc.columns]
# Mat.index = Mat.index.astype(int).astype(str)+'_cytof'
# Kepinuc = Kepinuc.append(Mat)
# # print(Kepinuc)
# # print(Kepinuc.T.corr())

# import matplotlib.pyplot as plt
# from matplotlib.pyplot import cm
# import seaborn as sns
# g=sns.clustermap(Kepinuc.T.corr(),cmap=plt.cm.seismic,
#                 #  vmin=amin,vmax=amax,
#                 # figsize=(10,20), annot_kws={"size":8}, center=0,
#                 figsize=(6, 10), annot_kws={"size":8}, center=0,
#                 annot=True, linewidths=1,linecolor='k',)
# g.ax_col_dendrogram.set_title('corr map') 
# plt.savefig('corr.png', format="png", bbox_inches="tight", pad_inches=0.2)

## Meandist

In [ ]:
if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist Epigen based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## sample percentage

In [ ]:
if visualize and not singleSample:
    arr =[]
    # full map
    ind = k.Clust != -1
    _ = drawUMAP_intVals(umapData,k,ind, 'by_sample',settings,
            title ='Tumor '+j+' Celliden UMAP', 
            Figname = name+'4_UMAP_(samples)')
    # map of each cluster
    clustNums = np.unique(k.Clust); clustNums = clustNums[clustNums!=-1]
    for clustNum in clustNums:
        ind = k.Clust == clustNum
        samplesPercentage = drawUMAP_intVals(umapData,k,ind ,'by_sample',settings,
            title =name+' UMAP clust' + str(clustNum), 
            Figname = name+'4_UMAP_clust'+str(clustNum)+'_(samples)')
        arr.append([clustNum,samplesPercentage])
    saveCsv(dir_plots,name,arr)

## features on UMAP

In [ ]:

if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor '+j+' Epigen UMAP', 
            Figname = name+'2_UMAP_')
    

## save data to file

In [ ]:
dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None

# ALL (Epigenetics + Cell identity Based)

In [ ]:

k = k_orig.copy()
CAll=pd.concat([k]).copy()

s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}

name = 'Tumor'+j+'_CellIden+EpiCols_'



### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['1'],s['n_neighbors']['1'] = 0.001,10
s['min_dist']['6'],s['n_neighbors']['6'] = 0.001,20
s['min_dist']['7'],s['n_neighbors']['7'] = 0.001,10
s['min_dist']['8'],s['n_neighbors']['8'] = 0.001,10
s['min_dist']['9'],s['n_neighbors']['9'] = 0.001,10
s['min_dist']['12456789'],s['n_neighbors']['12456789'] = 0.001,10
s['min_dist']['456789'],s['n_neighbors']['456789'] = 0.001,10

s['min_dist']['245']=0.001
s['n_neighbors']['245']=10
s['min_dist']['1245']=0.001
s['n_neighbors']['1245']=10

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]


umapData = calculate_umap(CAll[CellIden_EpiCols],n_neighbors, min_dist)
if visualize:
    drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')
    


# umap_params(k,MRK,dir_plots,
#             valA = min_dist, valB = n_neighbors,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.2, valB = 300,
#             )
# umap_params(k,MRK,dir_plots,
#             valA = 0.1, valB = 300,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.001, valB = 300,
#             )

### Dbscan and clusters

In [ ]:
s['min_samples']['1'],s['eps']['1']=50,0.19
s['min_samples']['6'],s['eps']['6']=50,0.19
s['min_samples']['7'],s['eps']['7']=80,0.15
s['min_samples']['8'],s['eps']['8']=50,0.15
s['min_samples']['9'],s['eps']['9']=50,0.08
s['min_samples']['12456789'],s['eps']['12456789']=50,0.1
s['min_samples']['456789'],s['eps']['456789']=70,0.1

s['min_samples']['245']=50
s['eps']['245']=0.19
s['min_samples']['1245']=65
s['eps']['1245']=0.12


min_samples = s['min_samples'][j]
eps = s['eps'][j]
dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps=eps,min_samples=min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
              
# dbscan_params(X_2d,dir_plots,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )
# l = labels==4
# s['min_samples']['1245']=70
# s['eps']['1245']=0.2
# dbscanData_,labels_,core_samples_mask_ = calculate_dbscan(umapData[l],eps=eps,min_samples=min_samples)
# labels[l] = labels_
# drawDbscan(dbscanData,labels,core_samples_mask,settings,
#               title=f'min_samples = {min_samples}, eps = {eps}',
#               figname = name+'1_dbscan')

In [ ]:
plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Epigen +Cell identity Based',
             figname = name+'1_Clusters')


## HeatMap by cluster

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
    
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Epigen+Cell Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)  


### HeatMap by sample

In [ ]:
if visualize and not singleSample:
    for group_name,group in zip(groups[0],groups[1]):
        group_name,group = groups[0][-1],groups[1][-1]
        HeatMap(k[k.Clust!=-1],group,settings,clustFeature='by_sample',
                title = 'T'+j+' Cell Iden Based: '+group_name+'by_sample',
                figname = name+'1_HeatMap_'+group_name+'by_sample')


## Meandist

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist Epigen+Cell based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## sample percentage

In [ ]:
if visualize and not singleSample:
    arr =[]
    # full map
    ind = k.Clust != -1
    _ = drawUMAP_intVals(umapData,k,ind, 'by_sample',settings,
            title ='Tumor '+j+' Celliden UMAP', 
            Figname = name+'4_UMAP_(samples)')
    # map of each cluster
    clustNums = np.unique(k.Clust); clustNums = clustNums[clustNums!=-1]
    for clustNum in clustNums:
        ind = k.Clust == clustNum
        samplesPercentage = drawUMAP_intVals(umapData,k,ind ,'by_sample',settings,
            title =name+' UMAP clust' + str(clustNum), 
            Figname = name+'4_UMAP_clust'+str(clustNum)+'_(samples)')
        arr.append([clustNum,samplesPercentage])
    saveCsv(dir_plots,name,arr)

## features on UMAP

In [ ]:

if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor '+j+' Epigen +Cell identity UMAP', 
            Figname = name+'2_UMAP_')
    

## save data to file

In [ ]:
dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None

# copy to single pdf

In [ ]:
dict ={}
dict['k'] = k_orig
# pickle_dump('k'+j+'_sample', dict,dir_data)

# imList2pdf(dir_plots,j,['CellIden','EpiCols','CellIden+EpiCols'])

end = time.process_time()
print ('total run time =', end-start )

In [ ]:
# imList2pdf(dir_plots,j,['CellIden','EpiCols','CellIden+EpiCols'])